In [ ]:
from fine import *
from modelling_datasets import *
from config import *
from modelling import * 

In [ ]:
import pandas as pd
import statsmodels.api as sm
import statsmodels.stats as sms
import numpy as np

In [ ]:
data = pd.read_csv('1. original_data/CPI/T10YIE.csv', index_col = 'DATE')
data['T10YIE'] = data['T10YIE'].replace('.', np.nan).astype(float) / 100
data = data.diff(1).iloc[1:].dropna()
data.index = pd.to_datetime(data.index)

In [ ]:
model = sm.OLS(
    yi, 
    sm.add_constant(X)
).fit(cov_type='HC3')
print(model.summary())

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
variance_inflation_factor(X)

# VIF dataframe 
vif_data = pd.DataFrame() 
vif_data["feature"] = X.columns 
  
# calculating VIF for each feature 
vif_data["VIF"] = [variance_inflation_factor(X.values, i) 
                          for i in range(len(X.columns))] 

vif_data

In [ ]:
ticker = 'AAPL.OQ'

models_types = list(regressors_sets.keys())[:6]
print(models_types)


fitted_models_list = []
for model_type in models_types:
    ### wo expected inflation
    
    X = data['T10YIE']
    yi = all_full_pct_return[ticker]

    yi = yi.loc[[date for date in X.index if date in yi.index]]
    yi = yi.rename(model_type)
    X = X.loc[yi.index]

    X = pd.concat([X, all_regressors[regressors_sets[model_type]]], axis = 1, join = 'inner')
    
    X.drop('T10YIE', axis = 1, inplace = True)
    
    model = sm.OLS(
        yi, 
        sm.add_constant(X)
    ).fit(cov_type='HC3')
    
    fitted_models_list.append(model)
    
    
    ### with expected inflation
    X = data['T10YIE']
    yi = all_full_pct_return[ticker]

    yi = yi.loc[[date for date in X.index if date in yi.index]]
    yi = yi.rename(f'{model_type} with Inflation')
    X = X.loc[yi.index]

    X = pd.concat([X, all_regressors[regressors_sets[model_type]]], axis = 1, join = 'inner')
    
    model = sm.OLS(
        yi, 
        sm.add_constant(X)
    ).fit(cov_type='HC3')
    
    fitted_models_list.append(model)

    
### Save as excel file
from statsmodels.iolib.summary2 import summary_col

summary_table = summary_col(fitted_models_list, stars=True, regressor_order = fitted_models_list[-1].params.index.tolist())
print(summary_table)
summary_table.tables[0].to_excel(f'Testing/{ticker} TIPS expected inflation in factor models.xlsx')

In [ ]:
all_full_pct_return

In [70]:



pvalues_list = []
noo_list = []
ticker_list = []
for ticker in tqdm(all_full_pct_return.columns):

    model_type = 'FF5C'
    # print(models_types)

    ### with expected inflation
    X = data['T10YIE']
    yi = all_full_pct_return[ticker]

    yi = yi.loc[[date for date in X.index if date in yi.index]]
    yi = yi.rename(f'{model_type} with Inflation').dropna()
    X = X.loc[yi.index]

    X = pd.concat([X, all_regressors[regressors_sets[model_type]]], axis = 1, join = 'inner')
    
    if X.shape[0] > 0:

        model = sm.OLS(
            yi, 
            sm.add_constant(X)
        ).fit(cov_type='HC1')

        pvalues_list.append(model.pvalues['T10YIE'])
        
        noo_list.append(X.shape[0])
        ticker_list.append(ticker)
    # print(ticker, yi.shape[0])

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1078/1078 [01:03<00:00, 17.11it/s]


In [76]:
p_values = pd.DataFrame([pvalues_list, noo_list], columns = ticker_list, index = ['P-values', 'noo']).T

In [77]:
p_values = p_values.sort_values(by = 'P-values', ascending=True)

In [78]:
p_values.to_excel('Testing/Pvalues.xlsx')

In [75]:
p_values[
    p_values['P-values'] < 0.1
]

,P-values,noo
NBR.N,9.249340e-18,4546.0
HP.N,2.303985e-15,4546.0
DNR.N^G20,3.457315e-15,4546.0
HAL.N,6.714093e-15,4546.0
PXD.N,1.011418e-14,4546.0
...,...,...
XL.N^I18,9.780409e-02,4546.0
SLR.N^J07,9.871667e-02,4546.0
HCA.N^K06,9.932764e-02,4546.0
MERQE.OQ^A06,9.939798e-02,4546.0
